In [121]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [122]:
import pandas as pd

file_path = '/content/drive/My Drive/Sports Modeling/mlb/mlb_proccessed_v1.csv'
df = pd.read_csv(file_path)

<ipython-input-122-63933633908f>:4: DtypeWarning: Columns (4,323,332,337,465,529,596,600,602,604,608,610,719,720,721,722,723,724,725,726) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [123]:
print(df.columns.tolist())


['id', 'status', 'scheduled', 'attendance', 'split_squad', 'final_inning', 'final_inning_half', 'home_name', 'home_runs', 'home_hits', 'home_errors', 'home_win', 'home_loss', 'home_scoring', 'home_hitting_overall_ab', 'home_hitting_overall_lob', 'home_hitting_overall_rbi', 'home_hitting_overall_abhr', 'home_hitting_overall_abk', 'home_hitting_overall_bip', 'home_hitting_overall_babip', 'home_hitting_overall_bbk', 'home_hitting_overall_bbpa', 'home_hitting_overall_iso', 'home_hitting_overall_obp', 'home_hitting_overall_ops', 'home_hitting_overall_seca', 'home_hitting_overall_slg', 'home_hitting_overall_xbh', 'home_hitting_overall_pitch_count', 'home_hitting_overall_lob_risp_2out', 'home_hitting_overall_team_lob', 'home_hitting_overall_ab_risp', 'home_hitting_overall_hit_risp', 'home_hitting_overall_rbi_2out', 'home_hitting_overall_linedrive', 'home_hitting_overall_groundball', 'home_hitting_overall_popup', 'home_hitting_overall_flyball', 'home_hitting_overall_ap', 'home_hitting_overall_

In [124]:
# List of weather-related columns
weather_columns = [
    'weather_forecast_obs_time', 'weather_forecast_temp_f', 'weather_forecast_condition',
    'weather_forecast_humidity', 'weather_forecast_dew_point_f', 'weather_forecast_cloud_cover',
    'weather_forecast_wind_speed_mph', 'weather_forecast_wind_direction',
    'weather_current_conditions_obs_time', 'weather_current_conditions_temp_f',
    'weather_current_conditions_condition', 'weather_current_conditions_humidity',
    'weather_current_conditions_dew_point_f', 'weather_current_conditions_cloud_cover',
    'weather_current_conditions_wind_speed_mph', 'weather_current_conditions_wind_direction'
]

# Filter rows where any of the weather columns are not null
filtered_df = df.dropna(subset=weather_columns, how='all')

# This keeps all columns in the filtered dataset
final_df = filtered_df

In [126]:
# Get the count of null values in each column
null_counts = final_df.isnull().sum()

# Filter the columns with null values
null_counts = null_counts[null_counts > 0]

# Display the columns with their respective null value counts
print(null_counts)

# Optionally, save the null counts to a CSV file for further analysis
null_counts.to_csv('null_value_counts.csv', header=['null_counts'])

# If you want to download the file, use the following code
from google.colab import files
files.download('null_value_counts.csv')

attendance                        896
split_squad                     12547
final_inning                      148
final_inning_half                 148
home_win                          565
                                ...  
home_probable_pitcher_suffix    14089
home_starting_pitcher_suffix    14089
pitching_loss_suffix            14013
pitching_win_suffix             14031
pitching_save_suffix            14071
Length: 716, dtype: int64


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Droping columns due to too many null values!

In [127]:
# Generate a list of columns with more than 4000 null values
columns_with_many_nulls = final_df.columns[final_df.isnull().sum() > 3000].tolist()

# Drop the identified columns
df = final_df.drop(columns=columns_with_many_nulls)

In [131]:
#Dropping more columns due to irrelevance
columns_irrelevance = ['id', 'status','home_scoring','away_scoring', 'officials', 'duration','pitching_win_status', 'pitching_loss_status', 'home_pitching_starters_era',
 'away_pitching_starters_era', 'away_pitching_bullpen_era','home_pitching_bullpen_era','time', 'weather_forecast_obs_time', 'weather_current_conditions_obs_time']

# Drop the identified columns
df = df.drop(columns=columns_irrelevance)

In [132]:
print(df.columns.tolist())

['scheduled', 'attendance', 'final_inning', 'final_inning_half', 'home_name', 'home_runs', 'home_hits', 'home_errors', 'home_win', 'home_loss', 'home_hitting_overall_ab', 'home_hitting_overall_lob', 'home_hitting_overall_rbi', 'home_hitting_overall_abhr', 'home_hitting_overall_abk', 'home_hitting_overall_bip', 'home_hitting_overall_babip', 'home_hitting_overall_bbk', 'home_hitting_overall_bbpa', 'home_hitting_overall_iso', 'home_hitting_overall_obp', 'home_hitting_overall_ops', 'home_hitting_overall_seca', 'home_hitting_overall_slg', 'home_hitting_overall_xbh', 'home_hitting_overall_pitch_count', 'home_hitting_overall_lob_risp_2out', 'home_hitting_overall_team_lob', 'home_hitting_overall_ab_risp', 'home_hitting_overall_hit_risp', 'home_hitting_overall_rbi_2out', 'home_hitting_overall_linedrive', 'home_hitting_overall_groundball', 'home_hitting_overall_popup', 'home_hitting_overall_flyball', 'home_hitting_overall_ap', 'home_hitting_overall_avg', 'home_hitting_overall_gofo', 'home_hittin

In [133]:
# Identify columns with null values
columns_with_nulls = df.columns[df.isnull().any()].tolist()

# Identify numeric columns with null values
numeric_columns_with_nulls = df.select_dtypes(include=['number']).columns[df.select_dtypes(include=['number']).isnull().any()].tolist()

# Step 1: Group-Based Mean Imputation (based on home_name and away_name) for numeric columns
for column in numeric_columns_with_nulls:
    df[column] = df.groupby(['home_name', 'away_name'])[column].transform(lambda x: x.fillna(x.mean()))

# Step 2: Overall Median Imputation for any remaining nulls in numeric columns
for column in numeric_columns_with_nulls:
    df[column].fillna(df[column].median(), inplace=True)

# Step 3: Overall Mode Imputation for any remaining nulls in non-numeric columns
non_numeric_columns_with_nulls = [col for col in columns_with_nulls if col not in numeric_columns_with_nulls]
for column in non_numeric_columns_with_nulls:
    df[column].fillna(df[column].mode()[0], inplace=True)

In [135]:
# Assuming NaN means 0 wins
df['home_win'].fillna(0, inplace=True)
df['away_win'].fillna(0, inplace=True)

# Calculate the previous game win counts
df['home_win_previous'] = df['home_win'].shift(1).fillna(0)
df['away_win_previous'] = df['away_win'].shift(1).fillna(0)

# Determine the win for each game
df['home_win_binary'] = ((df['home_win'] - df['home_win_previous']) > (df['away_win'] - df['away_win_previous'])).astype(int)
df['away_win_binary'] = ((df['away_win'] - df['away_win_previous']) > (df['home_win'] - df['home_win_previous'])).astype(int)

      home_win  away_win  home_win_previous  away_win_previous  \
5806       0.0       1.0                0.0                0.0   
5815       8.0      14.0                0.0                1.0   
5819      15.6      12.2                8.0               14.0   
5821      14.0      10.0               15.6               12.2   
5824      17.0      34.0               14.0               10.0   
5831       7.0      11.0               17.0               34.0   
5839      12.0      11.0                7.0               11.0   
5842       3.0       7.0               12.0               11.0   
5850       0.0       2.0                3.0                7.0   
5855      14.0      14.0                0.0                2.0   

      home_win_binary  away_win_binary  
5806                0                1  
5815                0                1  
5819                1                0  
5821                1                0  
5824                0                1  
5831                1      

<ipython-input-135-bd83c684cba5>:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['home_win_previous'] = df['home_win'].shift(1).fillna(0)
<ipython-input-135-bd83c684cba5>:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['away_win_previous'] = df['away_win'].shift(1).fillna(0)
<ipython-input-135-bd83c684cba5>:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) inste

In [136]:
# Check if home_win_binary and away_win_binary are the same
same_win_count = (df['home_win_binary'] == df['away_win_binary']).sum()

print("Count of games where home_win_binary and away_win_binary are the same:", same_win_count)


Count of games where home_win_binary and away_win_binary are the same: 515


In [137]:
# Remove rows where home_win_binary and away_win_binary are the same
df_filtered = df[df['home_win_binary'] != df['away_win_binary']]

# Print the count of removed rows
removed_rows_count = len(df) - len(df_filtered)
print("Number of removed rows where home_win_binary and away_win_binary are the same:", removed_rows_count)

Number of removed rows where home_win_binary and away_win_binary are the same: 515


In [138]:
# Create a new column 'winner_binary' based on 'home_win_binary' and 'away_win_binary'
df['winner_binary'] = df['home_win_binary']  # Since home_win_binary is 1 for home win and 0 for away win

# Drop the specified columns
columns_to_drop = ['home_win_previous', 'away_win_previous', 'home_win_binary', 'away_win_binary']
df.drop(columns=columns_to_drop, inplace=True)

# Print the DataFrame to verify the columns have been dropped
df['winner_binary']

<ipython-input-138-aa18f7bbf0f3>:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['winner_binary'] = df['home_win_binary']  # Since home_win_binary is 1 for home win and 0 for away win


,winner_binary
5806,0
5815,0
5819,1
5821,1
5824,0
...,...
21600,0
21601,1
21602,0
21603,1


## Feature Engineering

In [139]:
# Functions for calculating averages, recent performance, and head-to-head statistics
def calculate_average(df, group_col, target_col):
    return df.groupby(group_col)[target_col].transform('mean')

def calculate_recent_performance(df, group_col, target_col, window=5):
    return df.groupby(group_col)[target_col].transform(lambda x: x.rolling(window, min_periods=1).mean())

def calculate_head_to_head(df, group_col1, group_col2, target_col):
    return df.groupby([group_col1, group_col2])[target_col].transform('mean')

def calculate_win_streak(df, team_col, win_col):
    # Initialize the streak column with zeros
    streak = pd.Series(0, index=df.index)

    # Group by team and calculate the streak
    for team, group in df.groupby(team_col):
        streak[group.index] = group[win_col].groupby((group[win_col] != group[win_col].shift()).cumsum()).cumcount() + 1

    return streak
def create_features(df):
    # Basic team statistics
    df['home_avg_runs'] = calculate_average(df, 'home_name', 'home_runs')
    df['away_avg_runs'] = calculate_average(df, 'away_name', 'away_runs')
    df['home_avg_hits'] = calculate_average(df, 'home_name', 'home_hits')
    df['away_avg_hits'] = calculate_average(df, 'away_name', 'away_hits')
    df['home_avg_errors'] = calculate_average(df, 'home_name', 'home_errors')
    df['away_avg_errors'] = calculate_average(df, 'away_name', 'away_errors')

    # Recent performance
    df['home_recent_runs'] = calculate_recent_performance(df, 'home_name', 'home_runs')
    df['away_recent_runs'] = calculate_recent_performance(df, 'away_name', 'away_runs')
    df['home_recent_hits'] = calculate_recent_performance(df, 'home_name', 'home_hits')
    df['away_recent_hits'] = calculate_recent_performance(df, 'away_name', 'away_hits')
    df['home_recent_errors'] = calculate_recent_performance(df, 'home_name', 'home_errors')
    df['away_recent_errors'] = calculate_recent_performance(df, 'away_name', 'away_errors')

    # Head-to-head statistics
    df['head_to_head_runs'] = calculate_head_to_head(df, 'home_name', 'away_name', 'home_runs')

    # Derived features
    df['home_win_rate'] = df.groupby('home_name')['home_win'].transform(lambda x: x.rolling(10, min_periods=1).mean())
    df['away_win_rate'] = df.groupby('away_name')['away_win'].transform(lambda x: x.rolling(10, min_periods=1).mean())
    df['home_run_diff'] = df['home_runs'] - df['away_runs']
    df['home_hit_diff'] = df['home_hits'] - df['away_hits']
    df['home_error_diff'] = df['home_errors'] - df['away_errors']

    # Total and combined metrics
    df['total_runs'] = df['home_runs'] + df['away_runs']
    df['total_hits'] = df['home_hits'] + df['away_hits']
    df['total_errors'] = df['home_errors'] + df['away_errors']

    # Situational statistics
    df['home_risp'] = df['home_hitting_overall_ab_risp'] / df['home_hitting_overall_hit_risp']
    df['away_risp'] = df['away_hitting_overall_ab_risp'] / df['away_hitting_overall_hit_risp']
    df['home_two_out_rbi'] = df['home_hitting_overall_rbi_2out']
    df['away_two_out_rbi'] = df['away_hitting_overall_rbi_2out']

    # Advanced metrics
    df['home_iso'] = df['home_hitting_overall_iso']
    df['away_iso'] = df['away_hitting_overall_iso']
    df['home_obp'] = df['home_hitting_overall_obp']
    df['away_obp'] = df['away_hitting_overall_obp']
    df['home_slg'] = df['home_hitting_overall_slg']
    df['away_slg'] = df['away_hitting_overall_slg']

    # Ratios
    df['home_run_hit_ratio'] = df['home_runs'] / df['home_hits']
    df['away_run_hit_ratio'] = df['away_runs'] / df['away_hits']
    df['home_k_bb_ratio'] = df['home_pitching_overall_outcome_ktotal'] / df['home_pitching_overall_onbase_bb']
    df['away_k_bb_ratio'] = df['away_pitching_overall_outcome_ktotal'] / df['away_pitching_overall_onbase_bb']

    # Additional Features from previous steps

    # 1. Ratios and Differences
    df['home_away_win_ratio'] = df['home_pitching_overall_games_win'] / (df['away_pitching_overall_games_loss'] + 1)
    df['home_away_run_diff'] = df['home_hitting_overall_runs_total'] - df['away_pitching_overall_runs_total']
    df['home_away_era_diff'] = df['home_pitching_overall_era'] - df['away_pitching_overall_era']
    df['home_strikeout_to_walk_ratio'] = df['home_pitching_overall_k9'] / (df['home_pitching_overall_games_win'] + 1)

    # 2. Interaction Terms
    df['attendance_adjusted_performance'] = df['attendance'] * df['home_pitching_overall_games_win']
    df['slugging_percentage_interaction'] = df['home_hitting_overall_slg'] * df['away_pitching_overall_slg']

    # 3. Cumulative or Aggregated Features
    df['cumulative_home_runs'] = df['home_runs'].rolling(window=10, min_periods=1).sum()
    df['cumulative_wins'] = df['home_pitching_overall_games_win'].rolling(window=10, min_periods=1).sum()

    # 4. Custom Indices
    df['offensive_power_index'] = df['home_hitting_overall_ops'] + df['home_hitting_overall_rbi'] + df['home_hitting_overall_seca']
    df['defensive_efficiency_index'] = df['home_pitching_overall_whip'] + df['home_pitching_starters_outs_ktotal'] + df['home_pitching_overall_era']

    # 5. Rolling or Moving Averages
    df['rolling_avg_runs'] = df['home_hitting_overall_runs_total'].rolling(window=5, min_periods=1).mean()
    df['rolling_era'] = df['home_pitching_overall_era'].rolling(window=5, min_periods=1).mean()

    # 6. Weighted Metrics
    df['weighted_obp'] = df['home_hitting_overall_obp'] * df['home_hitting_overall_runs_total']
    df['weighted_strikeouts'] = df['home_pitching_starters_outs_ktotal'] / (df['home_pitching_starters_games_win'] + 1)

    # 7. Binary Flags
    df['high_attendance_flag'] = (df['attendance'] > 50000).astype(int)
    df['high_era_flag'] = (df['home_pitching_overall_era'] > 4.00).astype(int)

    # 9. Win Streaks or Momentum
    df['win_streak_length'] = calculate_win_streak(df, 'home_name', 'winner_binary')

    return df

# Example usage
df = create_features(df)


<ipython-input-139-c4f0e7d97a6e>:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['home_avg_runs'] = calculate_average(df, 'home_name', 'home_runs')
<ipython-input-139-c4f0e7d97a6e>:26: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['away_avg_runs'] = calculate_average(df, 'away_name', 'away_runs')
<ipython-input-139-c4f0e7d97a6e>:27: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using p

In [140]:
# Optionally, check if there are any NaNs introduced by the rolling or division operations
# Get the count of null values in each column
null_counts = df.isnull().sum()

# Filter the columns with null values
null_counts = null_counts[null_counts > 0]

# Display the columns with their respective null value counts
print(null_counts)

home_risp             135
away_risp             167
home_run_hit_ratio    158
away_run_hit_ratio    158
home_k_bb_ratio        13
away_k_bb_ratio        13
dtype: int64


In [141]:
# Identify columns with null values
columns_with_nulls = df.columns[df.isnull().any()].tolist()

# Identify numeric columns with null values
numeric_columns_with_nulls = df.select_dtypes(include=['number']).columns[df.select_dtypes(include=['number']).isnull().any()].tolist()

# Step 1: Group-Based Mean Imputation (based on home_name and away_name) for numeric columns
for column in numeric_columns_with_nulls:
    df[column] = df.groupby(['home_name', 'away_name'])[column].transform(lambda x: x.fillna(x.mean()))

# Step 2: Overall Median Imputation for any remaining nulls in numeric columns
for column in numeric_columns_with_nulls:
    df[column].fillna(df[column].median(), inplace=True)

# Step 3: Overall Mode Imputation for any remaining nulls in non-numeric columns
non_numeric_columns_with_nulls = [col for col in columns_with_nulls if col not in numeric_columns_with_nulls]
for column in non_numeric_columns_with_nulls:
    df[column].fillna(df[column].mode()[0], inplace=True)

In [142]:
# Optionally, check if there are any NaNs introduced by the rolling or division operations
# Get the count of null values in each column
null_counts = df.isnull().sum()

# Filter the columns with null values
null_counts = null_counts[null_counts > 0]

# Display the columns with their respective null value counts
print(null_counts)

Series([], dtype: int64)


In [143]:
# Convert the 'scheduled' column to datetime format
df['scheduled'] = pd.to_datetime(df['scheduled'])

# Extract relevant date and time features
df['year'] = df['scheduled'].dt.year
df['month'] = df['scheduled'].dt.month
df['day'] = df['scheduled'].dt.day
df['hour'] = df['scheduled'].dt.hour
df['minute'] = df['scheduled'].dt.minute
df['day_of_week'] = df['scheduled'].dt.dayofweek  # Monday=0, Sunday=6

# Optionally, you can drop the original 'scheduled' column if not needed
df = df.drop(columns=['scheduled'])

<ipython-input-143-27f28ace920a>:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['hour'] = df['scheduled'].dt.hour
<ipython-input-143-27f28ace920a>:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['minute'] = df['scheduled'].dt.minute
<ipython-input-143-27f28ace920a>:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `new

In [150]:
pregame_features = [ 'year', 'month', 'day', 'hour','minute','day_of_week',
    'home_name', 'away_name', 'weather_forecast_temp_f', 'weather_forecast_condition',
    'weather_forecast_wind_speed_mph', 'weather_forecast_wind_direction',
    'weather_forecast_humidity', 'weather_forecast_dew_point_f', 'weather_forecast_cloud_cover',
    'weather_current_conditions_temp_f','weather_current_conditions_condition', 'weather_current_conditions_humidity',
    'weather_current_conditions_dew_point_f', 'weather_current_conditions_cloud_cover',
    'weather_current_conditions_wind_speed_mph', 'weather_current_conditions_wind_direction'
]

all_features = [
    'year', 'month', 'day', 'hour','minute','day_of_week',
    'home_name', 'away_name', 'weather_forecast_temp_f', 'weather_forecast_condition', 'final_inning_half',
    'weather_forecast_wind_speed_mph', 'weather_forecast_wind_direction',
    'weather_forecast_humidity', 'weather_forecast_dew_point_f', 'weather_forecast_cloud_cover',
    'weather_current_conditions_temp_f','weather_current_conditions_condition', 'weather_current_conditions_humidity',
    'weather_current_conditions_dew_point_f', 'weather_current_conditions_cloud_cover',
    'weather_current_conditions_wind_speed_mph', 'weather_current_conditions_wind_direction',
    'home_avg_runs', 'away_avg_runs', 'home_avg_hits', 'away_avg_hits',
    'home_avg_errors', 'away_avg_errors', 'home_recent_runs', 'away_recent_runs',
    'home_recent_hits', 'away_recent_hits', 'home_recent_errors', 'away_recent_errors',
    'head_to_head_runs', 'home_win_rate', 'away_win_rate', 'home_run_diff',
    'home_hit_diff', 'home_error_diff', 'total_runs', 'total_hits', 'total_errors',
    'home_risp', 'away_risp', 'home_two_out_rbi', 'away_two_out_rbi', 'home_iso',
    'away_iso', 'home_obp', 'away_obp', 'home_slg', 'away_slg', 'home_run_hit_ratio',
    'away_run_hit_ratio', 'home_k_bb_ratio', 'away_k_bb_ratio',
    'home_away_win_ratio', 'home_away_run_diff', 'home_away_era_diff',
    'home_strikeout_to_walk_ratio', 'attendance_adjusted_performance',
    'slugging_percentage_interaction', 'cumulative_home_runs',
    'cumulative_wins', 'offensive_power_index', 'defensive_efficiency_index',
    'rolling_avg_runs', 'rolling_era', 'weighted_obp', 'weighted_strikeouts',
    'high_attendance_flag', 'high_era_flag', 'win_streak_length'
]


In [148]:
# List of columns that are not numerical
non_numerical_columns = df[pregame_features].select_dtypes(exclude=['number']).columns.tolist()

# Display the non-numerical columns
print(non_numerical_columns)

['home_name', 'away_name', 'weather_forecast_condition', 'weather_forecast_wind_direction', 'weather_current_conditions_condition', 'weather_current_conditions_wind_direction']


In [152]:
# Import necessary libraries
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import pandas as pd

# Initialize LabelEncoder
le = LabelEncoder()

# Convert non-numerical columns to numerical using Label Encoding
for col in ['home_name', 'away_name', 'weather_forecast_condition', 'final_inning_half',
            'weather_forecast_wind_direction', 'weather_current_conditions_condition',
            'weather_current_conditions_wind_direction']:
    df[col] = le.fit_transform(df[col])

In [153]:
from sklearn.model_selection import train_test_split

# Assuming your target variable is named 'target'
X_all = df[all_features]
X_pregame = df[pregame_features]
y = df['winner_binary']

# Split for the first model
X_all_train, X_all_test, y_train, y_test = train_test_split(X_all, y, test_size=0.3, random_state=42)


In [154]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam


In [ ]:
# Identify columns with infinity or very large values
columns_with_infinity = X_all_train.columns[np.isinf(X_all_train).any()]
columns_with_large_values = X_all_train.columns[(X_all_train.abs() > np.finfo(np.float64).max).any()]

# Combine and print the columns
problematic_columns = columns_with_infinity.union(columns_with_large_values)
print(f"Columns with infinity or very large values: {problematic_columns.tolist()}")

In [163]:
# columns with problematic values
problematic_columns = ['home_risp', 'away_risp', 'home_run_hit_ratio', 'home_k_bb_ratio', 'away_k_bb_ratio']

# Identify rows with infinity or very large values
rows_with_problems = X_all_train[problematic_columns].apply(lambda x: np.isinf(x) | (x.abs() > np.finfo(np.float64).max), axis=1)

# Remove rows with any problematic values
X_all_train_cleaned = X_all_train[~rows_with_problems.any(axis=1)]
y_train_cleaned = y_train[~rows_with_problems.any(axis=1)]

# Similarly clean the test set (if required)
rows_with_problems_test = X_all_test[problematic_columns].apply(lambda x: np.isinf(x) | (x.abs() > np.finfo(np.float64).max), axis=1)
X_all_test_cleaned = X_all_test[~rows_with_problems_test.any(axis=1)]
y_test_cleaned = y_test[~rows_with_problems_test.any(axis=1)]


## Comprehensive Model

In [190]:
# Standardizing features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_all_train_cleaned)
X_test_scaled = scaler.transform(X_all_test_cleaned)

# Define the comprehensive model
input_all = Input(shape=(len(all_features),))
x = Dense(128, activation='relu')(input_all)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(32, activation='relu')(x)
output_all = Dense(1, activation='sigmoid')(x)
comprehensive_model = Model(inputs=input_all, outputs=output_all)

# Compile the model
comprehensive_model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])

# Train the model
history = comprehensive_model.fit(X_train_scaled, y_train_cleaned, validation_data=(X_test_scaled, y_test_cleaned), epochs=20, batch_size=32)

# Evaluate the model
test_loss, test_acc = comprehensive_model.evaluate(X_test_scaled, y_test_cleaned)
print(f'Test Accuracy: {test_acc:.3f}')


Epoch 1/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.5278 - loss: 0.7257 - val_accuracy: 0.5866 - val_loss: 0.6736
Epoch 2/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5811 - loss: 0.6781 - val_accuracy: 0.6116 - val_loss: 0.6585
Epoch 3/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6013 - loss: 0.6621 - val_accuracy: 0.6257 - val_loss: 0.6457
Epoch 4/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6304 - loss: 0.6344 - val_accuracy: 0.6336 - val_loss: 0.6336
Epoch 5/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6253 - loss: 0.6465 - val_accuracy: 0.6370 - val_loss: 0.6263
Epoch 6/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6369 - loss: 0.6287 - val_accuracy: 0.6386 - val_loss: 0.6206
Epoch 7/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6521 - loss: 0.6224 - val_accuracy: 0.6457 - val_loss: 0.6170
Epoch 8/20
177/177 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.6604 - loss: 0.6084 - val_accuracy: 0.

## Transfer Learning

In [193]:
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model

# Define the prediction model with betting odds features
input_pred = Input(shape=(len(pregame_features),))
x_pred = Dense(128, activation='relu')(input_pred)
x_pred = Dropout(0.5)(x_pred)
x_pred = Dense(64, activation='relu')(x_pred)
x_pred = Dropout(0.5)(x_pred)
x_pred = Dense(32, activation='relu')(x_pred)
output_pred = Dense(1, activation='sigmoid')(x_pred)
prediction_model = Model(inputs=input_pred, outputs=output_pred)

# Transfer weights from the comprehensive model to the prediction model for the hidden layers
# Skipping the input layer and using the subsequent layers' weights

comprehensive_model_layers = [layer for layer in comprehensive_model.layers if 'dense' in layer.name or 'dropout' in layer.name]
prediction_model_layers = [layer for layer in prediction_model.layers if 'dense' in layer.name or 'dropout' in layer.name]

for comp_layer, pred_layer in zip(comprehensive_model_layers[1:], prediction_model_layers[1:]):
    pred_layer.set_weights(comp_layer.get_weights())

# Compile the prediction model
prediction_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [194]:
# Train the model
from sklearn.metrics import accuracy_score

# Assuming X_pregame contains the features for pregame analysis and y contains the labels
X_pregame_train, X_pregame_test, y_train_pregame, y_test_pregame = train_test_split(X_pregame, y, test_size=0.3, random_state=42)

history_pregame = prediction_model.fit(X_pregame_train, y_train_pregame, validation_data=(X_pregame_test, y_test_pregame), epochs=10, batch_size=32)

# Make predictions
predictions = prediction_model.predict(X_pregame_test)

# Convert probabilities to binary predictions
predictions_binary = (predictions > 0.5).astype(int)

# Calculate accuracy
accuracy = accuracy_score(y_test_pregame, predictions_binary)
print(f'Test Accuracy: {accuracy:.3f}')

Epoch 1/10
309/309 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - accuracy: 0.5061 - loss: 17.2935 - val_accuracy: 0.5155 - val_loss: 0.7137
Epoch 2/10
309/309 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5018 - loss: 0.9242 - val_accuracy: 0.5155 - val_loss: 0.6979
Epoch 3/10
309/309 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5075 - loss: 0.7829 - val_accuracy: 0.5155 - val_loss: 0.7125
Epoch 4/10
309/309 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5031 - loss: 0.7183 - val_accuracy: 0.5155 - val_loss: 0.7095
Epoch 5/10
309/309 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.5121 - loss: 0.7179 - val_accuracy: 0.5155 - val_loss: 0.7081
Epoch 6/10
309/309 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.5145 - loss: 0.7001 - val_accuracy: 0.5155 - val_loss: 0.7048
Epoch 7/10
309/309 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5180 - loss: 0.6998 - val_accuracy: 0.5155 - val_loss: 0.7050
Epoch 8/10
309/309 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5247 - loss: 0.6981 - val_accuracy: 0